# Generate artificial data sets
Here, we generate data sets to explore the behavior of scores describing differences between high-dimensional point clouds in presence of noise and outliers.  

In [ ]:
import numpy as np

In [ ]:
# Number of points in control dataset
NR = 2000
# Number of points in other datasets
N = 1000
# Number of dimensions in each dataset
D = 100

## Dataset 1 - Reference R
We assume our data of interest to follow a multivariate normal distribution: In a morphological profiling, components are to some extent *independent (by removing correlated morphological features) and* normally distributed (by using a log-transformation).

In [ ]:
np.random.seed(1)

### Data center

In [ ]:
# The reference is centered on 0
refMu = [0]*D

### Data covariance

In [ ]:
# Diagonal: variances follow a Beta distribution of shape and scale parameters equal to 2 and 2
# Rationale: Some variability in scales with some high values, and no negative values
sigma_diag = np.eye(D)
sigma_diag = sigma_diag * np.random.gamma(2,2,D)

The following may lead to non positive-semidefinite matrices:

```
# Lower and upper triangles: some relation but no strong covariances
# Rationale: in a morphological profiling, we remove highly correlated features
# E.g. for a correlation threshold of 0.5, since cov(x,y) = corr(x,y) * sigma(x) * sigma(y)
# This corresponds to a covariance threshold of 0.5 * sigma(x) * sigma(y) which is 2 in average
# We select a centered normal distribution of variance 0.75 based on the 99-th percentile
sigma_tril = np.random.normal(0, 0.75, [D,D])
sigma_tril = np.tril(sigma_tril ) + np.tril(sigma_tril , -1).T
# Possible improvement: rescale all values based on the actual standard deviations of the features studied 
# (instead of using the expectation)
```

In [ ]:
sigma_tril = np.random.normal(0, 0.05, [D,D])
sigma_tril = np.tril(sigma_tril ) + np.tril(sigma_tril , -1).T

In [ ]:
refSigma = sigma_diag + sigma_tril - np.eye(D)*np.diag(sigma_tril)

In [ ]:
# Matrix is symmetrical
assert all([refSigma[i,j] == refSigma[j,i] for i in range(D) for j in range(D) if j>i])
# Sylvester's criterion of positive semidefinite matrices
assert all([np.linalg.det(refSigma[:(size+1),:(size+1)]) > 0 for size in range(D)])

In [ ]:
matR = np.random.multivariate_normal(refMu, refSigma, NR)

In [ ]:
np.savetxt("Data/matR.csv", matR, delimiter=",")

## Dataset 2 - Negative control N
This is generated with the same generator as the positive control

In [ ]:
np.random.seed(2)

### Data covariance

In [ ]:
matN = np.random.multivariate_normal(refMu, refSigma, N)

In [ ]:
np.savetxt("Data/matN.csv", matN, delimiter=",")

## Dataset 3 - Positive control (shifted) PS

In [ ]:
np.random.seed(3)

### Data center

In [ ]:
# The reference is not centered on 0 anymore
mu = np.random.normal(0, 1, D)

In [ ]:
matPS = np.random.multivariate_normal(mu, refSigma, N)

In [ ]:
np.savetxt("Data/matPS.csv", matPS, delimiter=",")

## Dataset 4 - Positive control (reshaped) PR
This is generated with the same parameters but a different generator as the reference data set, so the covariance of is different.

In [ ]:
np.random.seed(4)

### Data covariance

In [ ]:
# Diagonal: variances follow a Beta distribution of shape and scale parameters equal to 2 and 2
# Rationale: Some variability in scales with some high values, and no negative values
sigma_diag = np.eye(D)
sigma_diag = sigma_diag * np.random.gamma(2,2,D)

In [ ]:
sigma_tril = np.random.normal(0, 0.05, [D,D])
sigma_tril = np.tril(sigma_tril ) + np.tril(sigma_tril , -1).T

In [ ]:
sigma = sigma_diag + sigma_tril - np.eye(D)*np.diag(sigma_tril)

In [ ]:
# Matrix is symmetrical
assert all([sigma[i,j] == sigma[j,i] for i in range(D) for j in range(D) if j>i])
# Sylvester's criterion of positive semidefinite matrices
assert all([np.linalg.det(sigma[:(size+1),:(size+1)]) > 0 for size in range(D)])

In [ ]:
matPR = np.random.multivariate_normal(refMu, sigma, N)

In [ ]:
np.savetxt("Data/matPR.csv", matPR, delimiter=",")

## Dataset 5 - Uniform noise U

In [ ]:
np.random.seed(5)

In [ ]:
matU = np.random.multivariate_normal(refMu, refSigma, N) + np.random.uniform(-1,1, size = [N,D])

In [ ]:
np.savetxt("Data/matU.csv", matU, delimiter=",")

## Dataset 6 - Negative with few outliers NF

In [ ]:
np.random.seed(6)

In [ ]:
matNF = np.random.multivariate_normal(refMu, refSigma, N)

In [ ]:
print(matNF.mean())
print(np.median(matNF))
print(np.std(matNF))

In [ ]:
# For 1 out of k points, multiply 1 out of k features by 2
for iOutlier in np.random.choice(N, round(N/10), replace = False):
    matNF[iOutlier,np.random.choice(D, round(D/10), replace = False)] *= 2

In [ ]:
print(matNF.mean())
print(np.median(matNF))
print(np.std(matNF))

In [ ]:
np.savetxt("Data/matNF.csv", matNF, delimiter=",")

## Dataset 7 - Negative with many outliers NM

In [ ]:
np.random.seed(7)

In [ ]:
matNM = np.random.multivariate_normal(refMu, refSigma, N)

In [ ]:
print(matNM.mean())
print(np.median(matNM))
print(np.std(matNM))

In [ ]:
# For 1 out of k points, multiply 1 out of k features by 2
for iOutlier in np.random.choice(N, round(N/3), replace = False):
    matNM[iOutlier,np.random.choice(D, round(D/3), replace = False)] *= 2

In [ ]:
print(matNM.mean())
print(np.median(matNM))
print(np.std(matNM))

In [ ]:
np.savetxt("Data/matNM.csv", matNM, delimiter=",")

## Dataset 8 - Positive with few outliers PF

In [ ]:
np.random.seed(8)

### Data center

In [ ]:
# The reference is not centered on 0 anymore
mu = np.random.normal(0, 1, D)

In [ ]:
matPF = np.random.multivariate_normal(mu, refSigma, N)

In [ ]:
print(matPF.mean())
print(np.median(matPF))
print(np.std(matPF))

In [ ]:
# For 1 out of k points, multiply 1 out of k features by 2
for iOutlier in np.random.choice(N, round(N/10), replace = False):
    matPF[iOutlier,np.random.choice(D, round(D/10), replace = False)] *= 2

In [ ]:
print(matPF.mean())
print(np.median(matPF))
print(np.std(matPF))

In [ ]:
np.savetxt("Data/matPF.csv", matPF, delimiter=",")

## Dataset 9 - Positive with many outliers PM

In [ ]:
np.random.seed(9)

### Data center

In [ ]:
# The reference is not centered on 0 anymore
mu = np.random.normal(0, 1, D)

In [ ]:
matPM = np.random.multivariate_normal(mu, refSigma, N)

In [ ]:
print(matPM.mean())
print(np.median(matPM))
print(np.std(matPM))

In [ ]:
# For 1 out of k points, multiply 1 out of k features by 2
for iOutlier in np.random.choice(N, round(N/3), replace = False):
    matPM[iOutlier,np.random.choice(D, round(D/3), replace = False)] *= 2

In [ ]:
print(matPM.mean())
print(np.median(matPM))
print(np.std(matPM))

In [ ]:
np.savetxt("Data/matPM.csv", matPM, delimiter=",")